In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.cuda.is_available()

True

In [2]:
from src.vlstm_v3.interface import qkvkernel

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt21cu118/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt21cu118/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt21cu118/lib


Using /home/max/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu118/vlstm_v3/build.ninja...
Building extension module vlstm_v3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_v3...


### qkvtest

In [3]:
S = 8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 4 # dim per head
DTYPE = torch.float16
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
qs, qs.shape

(tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
           [0.3999, 0.5000, 0.6001, 0.7002],
           [0.7998, 0.8999, 1.0000, 1.0996],
           [1.2002, 1.2998, 1.4004, 1.5000],
           [1.5996, 1.7002, 1.7998, 1.9004],
           [2.0000, 2.0996, 2.1992, 2.3008],
           [2.4004, 2.5000, 2.5996, 2.6992],
           [2.8008, 2.9004, 3.0000, 3.0996]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 8, 4]))

In [5]:
rs = qs @ ks.transpose(-1, -2)
rs, rs.shape

(tensor([[[[ 0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,  0.6001,
             0.6001],
           [ 2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,  2.1992,
             2.1992],
           [ 3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,  3.7988,
             3.7988],
           [ 5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,  5.3984,
             5.3984],
           [ 7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,  7.0000,
             7.0000],
           [ 8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,  8.6016,
             8.6016],
           [10.2031, 10.2031, 10.2031, 10.2031, 10.2031, 10.2031, 10.2031,
            10.2031],
           [11.7969, 11.7969, 11.7969, 11.7969, 11.7969, 11.7969, 11.7969,
            11.7969]]]], device='cuda:0', dtype=torch.float16),
 torch.Size([1, 1, 8, 8]))

In [6]:
# pytorch version
rs = qs @ ks.transpose(-1, -2) @ vs
rs, rs.shape

(tensor([[[[ 4.8008,  4.8008,  4.8008,  4.8008],
           [17.5938, 17.5938, 17.5938, 17.5938],
           [30.3906, 30.3906, 30.3906, 30.3906],
           [43.1875, 43.1875, 43.1875, 43.1875],
           [56.0000, 56.0000, 56.0000, 56.0000],
           [68.8125, 68.8125, 68.8125, 68.8125],
           [81.6250, 81.6250, 81.6250, 81.6250],
           [94.3750, 94.3750, 94.3750, 94.3750]]]], device='cuda:0',
        dtype=torch.float16),
 torch.Size([1, 1, 8, 4]))

In [ ]:
# cuda kernel
rs = qkvkernel(mat_Q=qs, mat_K=ks, mat_V=vs)
rs, rs.shape

: 

In [7]:
qs

tensor([[[[0.0000, 0.1000, 0.2000, 0.3000],
          [0.3999, 0.5000, 0.6001, 0.7002],
          [0.7998, 0.8999, 1.0000, 1.0996],
          [1.2002, 1.2998, 1.4004, 1.5000],
          [1.5996, 1.7002, 1.7998, 1.9004],
          [2.0000, 2.0996, 2.1992, 2.3008],
          [2.4004, 2.5000, 2.5996, 2.6992],
          [2.8008, 2.9004, 3.0000, 3.0996]]]], device='cuda:0',
       dtype=torch.float16)

### Matmul kernel Test from vlstm_v2

In [9]:
# from src.vlstm_v3.interface import testkernel, copykernel, mmkernelv1

In [ ]:
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")

In [11]:
S = 8 # sequence length
DH = 8 # hidden size

In [12]:
matA = torch.arange((2*S * DH), device=DEVICE, dtype=DTYPE).reshape((2*S, DH))
matB = torch.ones((DH, S), device=DEVICE, dtype=DTYPE)
matA.shape, matB.shape, matA, matA.sum(-1)

(torch.Size([16, 8]),
 torch.Size([8, 8]),
 tensor([[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.],
         [  8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.],
         [ 16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.],
         [ 24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.],
         [ 32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
         [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.],
         [ 48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.],
         [ 56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.],
         [ 64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.],
         [ 72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
         [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.],
         [ 88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.],
         [ 96.,  97.,  98.,  99., 100., 101., 102., 103.],
         [104., 105., 106., 107., 108., 109., 110., 111.],
         [112., 113., 114., 115., 116., 117., 118., 119.],
         [120., 121., 122., 123., 124., 125., 126., 127.]], device='cuda

In [13]:
# pytorch
pt_out = matA @ matB
pt_out, pt_out.shape

(tensor([[ 28.,  28.,  28.,  28.,  28.,  28.,  28.,  28.],
         [ 92.,  92.,  92.,  92.,  92.,  92.,  92.,  92.],
         [156., 156., 156., 156., 156., 156., 156., 156.],
         [220., 220., 220., 220., 220., 220., 220., 220.],
         [284., 284., 284., 284., 284., 284., 284., 284.],
         [348., 348., 348., 348., 348., 348., 348., 348.],
         [412., 412., 412., 412., 412., 412., 412., 412.],
         [476., 476., 476., 476., 476., 476., 476., 476.],
         [540., 540., 540., 540., 540., 540., 540., 540.],
         [604., 604., 604., 604., 604., 604., 604., 604.],
         [668., 668., 668., 668., 668., 668., 668., 668.],
         [732., 732., 732., 732., 732., 732., 732., 732.],
         [796., 796., 796., 796., 796., 796., 796., 796.],
         [860., 860., 860., 860., 860., 860., 860., 860.],
         [924., 924., 924., 924., 924., 924., 924., 924.],
         [988., 988., 988., 988., 988., 988., 988., 988.]], device='cuda:0',
        dtype=torch.bfloat16),
 torch.

In [14]:
matA.is_contiguous(), matB.is_contiguous()

(True, True)

In [15]:
cu_out = mmkernelv1(mat_A=matA, mat_B=matB)
cu_out, cu_out.shape

NameError: name 'mmkernelv1' is not defined

In [ ]:
matA[9]

tensor([72., 73., 74., 75., 76., 77., 78., 79.], device='cuda:0',
       dtype=torch.bfloat16)

In [ ]:
matA[9].cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 448., 528., 608.], device='cuda:0',
       dtype=torch.bfloat16)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cuda:0')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 448., 528., 608.], device='cuda:0',
       dtype=torch.bfloat16)

In [ ]:
torch.arange(72,80, dtype=torch.bfloat16, device=torch.device('cpu')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 448., 524., 604.], dtype=torch.bfloat16)

In [ ]:
torch.arange(72,80, dtype=torch.float32, device=torch.device('cpu')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 447., 525., 604.])

In [ ]:
torch.arange(72,80, dtype=torch.float16, device=torch.device('cuda:0')).cumsum(-1)

tensor([ 72., 145., 219., 294., 370., 447., 525., 604.], device='cuda:0',
       dtype=torch.float16)

In [ ]:
# cu_out = mmkernelv2(mat_A=matA, mat_B=matB)
# cu_out, cu_out.shape

In [ ]:
# mat @ mat.T @ mat

### pytorch version

In [ ]:
H = 6 # hidden size
S = 5 # seq len
B = 1 # batch size
NH = 2 # num heads
DH = H // NH # dim per head
DTYPE = torch.bfloat16
DEVICE = torch.device("cuda:0")
assert H % NH == 0

In [ ]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ds = torch.rand((B, NH, S, S), device=DEVICE, dtype=DTYPE)

max_log_D, _ = torch.max(ds.view(B, NH, -1), dim=-1, keepdim=True)  # (B, NH, 1)
log_D_matrix_stabilized = ds - max_log_D.unsqueeze(-1)  # (B, NH, S, S) = (B, NH, S, S) - (B, NH, 1, 1)
D_matrix = torch.exp(log_D_matrix_stabilized)  # (B, NH, S, S)
mval = torch.exp(-max_log_D.unsqueeze(-1))